In [1]:
#! pip install https://github.com/pandas-profiling/pandas-profiling/archive/master.zip
# then restart kernel 

In [2]:
import numpy as np
import pandas as pd
import glob
import os
import re
#from google.colab import files


def load_games():
  seasons = ['15-16.csv','16-17.csv','17-18.csv','18-19.csv','19-20.csv']   #in case of google colab use files.upload()
  dfs = []

  for filename in seasons:

    dfs.append(pd.read_csv(filename, encoding='utf8', names=['Home team', 'Away team', 'Board no. ', 'Title home', 'Elo home',
                                                          'Player home','Color home', 'Result', 'Color away',
                                                          'Player away','Elo away','Title away', 'Date', 'Time', 'League', 'Round', 'Season']) )

  #Concatenate all data into one DataFrame

  all_games = pd.concat(dfs, ignore_index=True)
  return all_games


df = load_games()

df


,Home team,Away team,Board no.,Title home,Elo home,Player home,Color home,Result,Color away,Player away,Elo away,Title away,Date,Time,League,Round,Season
0,HÜSI SC,HVSE-Infraplan,1,IM,2374,Böröcz István,black,01,white,Jankovic Alojzije,2557,GM,2015-11-07,15:00,nb1,0,15-16
1,HÜSI SC,HVSE-Infraplan,2,NaN,2362,Havasi Gergő,white,½½,black,Ruck Róbert,2565,GM,2015-11-07,15:00,nb1,0,15-16
2,HÜSI SC,HVSE-Infraplan,3,GM,2470,Kosic Dragan,black,½½,white,Németh Miklós,2469,IM,2015-11-07,15:00,nb1,0,15-16
3,HÜSI SC,HVSE-Infraplan,4,IM,2424,Stankovic Milos,white,½½,black,Dr. Kovács Gábor,2462,IM,2015-11-07,15:00,nb1,0,15-16
4,HÜSI SC,HVSE-Infraplan,5,GM,2426,Drazic Sinisa,black,½½,white,Mészáros Michal,2389,IM,2015-11-07,15:00,nb1,0,15-16
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27975,Cs1,Cs2,2,NaN,2100,Kovács Apollón,white,10\r\n,black,Elek Mónika,1869,NaN,2020-09-13,10:00,nb2toth,8,19-20
27976,Cs1,Cs2,1,NaN,2118,Károlyfalvi Zoltán,black,\r\n10,white,Markó Árpád,1948,NaN,2020-09-13,10:00,nb2toth,8,19-20
27977,Cs1,Cs2,2,NaN,2100,Kovács Apollón,white,10\r\n,black,Elek Mónika,1869,NaN,2020-09-13,10:00,nb2toth,8,19-20
27978,Cs1,Cs2,1,NaN,2118,Károlyfalvi Zoltán,black,\r\n10,white,Markó Árpád,1948,NaN,2020-09-13,10:00,nb2toth,8,19-20


In [3]:
def correct_round(df):
  
  df['Round'] = df['Round'].apply(np.int64 )
  #df['Round'] = df['Round'].apply(lambda x: x+1 )
  return df

df = correct_round(df)
df

def drop_no_team_row(df):
  df = df[(df['Home team'] != 'Cs1') & (df['Home team'] != 'Cs2')]
  return df


def convert_result(df):
    
    df['Result_home'] = df['Result'].apply(lambda x: 0 if x == '01' else (0.5 if x == '½½' else 1))
    df['Result_away'] = df['Result'].apply(lambda x: 1 if x == '01' else (0.5 if x == '½½' else 0))


    del df['Result']
    df.reindex(df.columns, axis=1)
    return df

df = drop_no_team_row(df)
df = convert_result(df)
df

<ipython-input-3-6c61c497ca09>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Result_home'] = df['Result'].apply(lambda x: 0 if x == '01' else (0.5 if x == '½½' else 1))
<ipython-input-3-6c61c497ca09>:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Result_away'] = df['Result'].apply(lambda x: 1 if x == '01' else (0.5 if x == '½½' else 0))


,Home team,Away team,Board no.,Title home,Elo home,Player home,Color home,Color away,Player away,Elo away,Title away,Date,Time,League,Round,Season,Result_home,Result_away
0,HÜSI SC,HVSE-Infraplan,1,IM,2374,Böröcz István,black,white,Jankovic Alojzije,2557,GM,2015-11-07,15:00,nb1,0,15-16,0.0,1.0
1,HÜSI SC,HVSE-Infraplan,2,NaN,2362,Havasi Gergő,white,black,Ruck Róbert,2565,GM,2015-11-07,15:00,nb1,0,15-16,0.5,0.5
2,HÜSI SC,HVSE-Infraplan,3,GM,2470,Kosic Dragan,black,white,Németh Miklós,2469,IM,2015-11-07,15:00,nb1,0,15-16,0.5,0.5
3,HÜSI SC,HVSE-Infraplan,4,IM,2424,Stankovic Milos,white,black,Dr. Kovács Gábor,2462,IM,2015-11-07,15:00,nb1,0,15-16,0.5,0.5
4,HÜSI SC,HVSE-Infraplan,5,GM,2426,Drazic Sinisa,black,white,Mészáros Michal,2389,IM,2015-11-07,15:00,nb1,0,15-16,0.5,0.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27955,Félegyháza-Csongrád,Gyulai SE,8,NaN,1924,Kelemen Sándor,white,black,Oltean Cristian-Daniel,1911,NaN,2020-03-08,10:00,nb2toth,6,19-20,0.0,1.0
27956,Félegyháza-Csongrád,Gyulai SE,9,NaN,1943,Szekeres Sebestyén,black,white,Györgydeák Tibor,1766,NaN,2020-03-08,10:00,nb2toth,6,19-20,0.5,0.5
27957,Félegyháza-Csongrád,Gyulai SE,10,NaN,1738,Hideg Ferenc Bendegúz,white,black,Gurzó Bence Bendeguz,1571,NaN,2020-03-08,10:00,nb2toth,6,19-20,0.5,0.5
27958,Félegyháza-Csongrád,Gyulai SE,11,NaN,1590,Nagy Noémi,black,white,Vaszkó Szabolcs,NaN,-,2020-03-08,10:00,nb2toth,6,19-20,1.0,0.0


In [4]:
def correct_columns(df1):
  #df1 = df.copy()
  df['Elo home'] = df['Elo home'].str.replace(r'\D+','1400')
  df['Elo home'] = df['Elo home'].fillna(1400)
  #df['Elo home'] = df['Elo home'].apply(lambda x: 1400 if pd.isnull(x) else int(x))
  #df['Elo home'] = df['Elo home'].apply(np.int64 )

  df['Elo away'] = df['Elo away'].str.replace(r'\D+','1400')
  df['Elo away'] = df['Elo away'].fillna(1400)
  #df['Elo away'] = df['Elo away'].apply(lambda x: 1400 if pd.isnull(x) else int(x))
  #df['Elo away'] = df['Elo away'].apply(np.int64 )

  df['Title home'] = df['Title home'].apply(lambda x: 'No Title' if pd.isnull(x) else x)
  df['Title away'] = df['Title away'].apply(lambda x: 'No Title' if pd.isnull(x) else x ) 

  df['Color home'] = df['Color home'].apply(lambda x: 0 if x == 'white' else 1)
  df['Color away'] = df['Color away'].apply(lambda x: 0 if x == 'white' else 1)


  return df 




df = correct_columns(df)
df

<ipython-input-4-ac8f996933b8>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Elo home'] = df['Elo home'].str.replace(r'\D+','1400')
<ipython-input-4-ac8f996933b8>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Elo home'] = df['Elo home'].fillna(1400)
<ipython-input-4-ac8f996933b8>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/p

,Home team,Away team,Board no.,Title home,Elo home,Player home,Color home,Color away,Player away,Elo away,Title away,Date,Time,League,Round,Season,Result_home,Result_away
0,HÜSI SC,HVSE-Infraplan,1,IM,2374,Böröcz István,1,0,Jankovic Alojzije,2557,GM,2015-11-07,15:00,nb1,0,15-16,0.0,1.0
1,HÜSI SC,HVSE-Infraplan,2,No Title,2362,Havasi Gergő,0,1,Ruck Róbert,2565,GM,2015-11-07,15:00,nb1,0,15-16,0.5,0.5
2,HÜSI SC,HVSE-Infraplan,3,GM,2470,Kosic Dragan,1,0,Németh Miklós,2469,IM,2015-11-07,15:00,nb1,0,15-16,0.5,0.5
3,HÜSI SC,HVSE-Infraplan,4,IM,2424,Stankovic Milos,0,1,Dr. Kovács Gábor,2462,IM,2015-11-07,15:00,nb1,0,15-16,0.5,0.5
4,HÜSI SC,HVSE-Infraplan,5,GM,2426,Drazic Sinisa,1,0,Mészáros Michal,2389,IM,2015-11-07,15:00,nb1,0,15-16,0.5,0.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27955,Félegyháza-Csongrád,Gyulai SE,8,No Title,1924,Kelemen Sándor,0,1,Oltean Cristian-Daniel,1911,No Title,2020-03-08,10:00,nb2toth,6,19-20,0.0,1.0
27956,Félegyháza-Csongrád,Gyulai SE,9,No Title,1943,Szekeres Sebestyén,1,0,Györgydeák Tibor,1766,No Title,2020-03-08,10:00,nb2toth,6,19-20,0.5,0.5
27957,Félegyháza-Csongrád,Gyulai SE,10,No Title,1738,Hideg Ferenc Bendegúz,0,1,Gurzó Bence Bendeguz,1571,No Title,2020-03-08,10:00,nb2toth,6,19-20,0.5,0.5
27958,Félegyháza-Csongrád,Gyulai SE,11,No Title,1590,Nagy Noémi,1,0,Vaszkó Szabolcs,1400,-,2020-03-08,10:00,nb2toth,6,19-20,1.0,0.0


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 27898 entries, 0 to 27959
Data columns (total 18 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Home team    27898 non-null  object 
 1   Away team    27898 non-null  object 
 2   Board no.    27898 non-null  int64  
 3   Title home   27898 non-null  object 
 4   Elo home     27898 non-null  object 
 5   Player home  27895 non-null  object 
 6   Color home   27898 non-null  int64  
 7   Color away   27898 non-null  int64  
 8   Player away  27894 non-null  object 
 9   Elo away     27898 non-null  object 
 10  Title away   27898 non-null  object 
 11  Date         27898 non-null  object 
 12  Time         27898 non-null  object 
 13  League       27898 non-null  object 
 14  Round        27898 non-null  int64  
 15  Season       27898 non-null  object 
 16  Result_home  27898 non-null  float64
 17  Result_away  27898 non-null  float64
dtypes: float64(2), int64(4), object(12)
memory usa

In [6]:
import csv
import os

def load_rankingLists():
  
    ranklists = [] 
    dfs = []
    
    keyword = 'nb'   #in case of google colab just use files.upload()
    for filename in os.listdir('.'):
        if keyword in filename and filename.startswith("1") and filename.endswith(".csv"):
            ranklists.append(filename)

    


    for filename in ranklists:
        

    
        with open(filename, encoding="utf-8") as f:
            reader = csv.reader(f, delimiter=',', skipinitialspace=True)
            next(reader)
            num_col = 0 
            while(num_col == 0 ):
                #print("next reader")
                num_col= len(next(reader))



        season = re.compile(r'(\d\d-\d\d)').match(filename).group(0)
        league = re.compile(r'\d\d-\d\d(.+)').match(filename).group(1)
        league = league[:-4]
        
        
        
        full_columns = ['Index','Player name', 'Youth','Female', 'Nationality', 'Title', 'Elo',
                                                              'Round 1','Round 2', 'Round 3', 'Round 4','Round 5','Round 6','Round 7', 'Round 8', 'Round 9', 'Round 10', 'Round 11', 'Total',
                                                              'Born year', 'Team name', 'Standard elo', 'Rapid elo', 'Blitz elo',
                                                              'Game no 2021-Feb standard','Game no 2021-Feb rapid', 'Game no 2021-Feb blitz',
                                                             'Game no 2021-Jan standard','Game no 2021-Jan rapid', 'Game no 2021-Jan blitz',
                                                             'Game no 2020-Dec standard','Game no 2020-Dec rapid', 'Game no 2020-Dec blitz',
                                                             'Game no 2020-Nov standard','Game no 2020-Nov rapid', 'Game no 2020-Nov blitz',
                                                             'Game no 2020-Oct standard','Game no 2020-Oct rapid', 'Game no 2020-Oct blitz',
                                                             'Game no 2020-Sep standard','Game no 2020-Sep rapid', 'Game no 2020-Sep blitz',
                                                             'Game no 2020-Aug standard','Game no 2020-Aug rapid', 'Game no 2020-Aug blitz',
                                                             'Game no 2020-Jul standard','Game no 2020-Jul rapid', 'Game no 2020-Jul blitz',
                                                             'Game no 2020-Jun standard','Game no 2020-Jun rapid', 'Game no 2020-Jun blitz',
                                                             'Game no 2020-May standard','Game no 2020-May rapid', 'Game no 2020-May blitz',
                                                             'Game no 2020-Apr standard','Game no 2020-Apr rapid', 'Game no 2020-Apr blitz',
                                                             'Game no 2020-Mar standard','Game no 2020-Mar rapid', 'Game no 2020-Mar blitz',
                                                             'Game no 2020-Feb standard','Game no 2020-Feb rapid', 'Game no 2020-Feb blitz',
                                                             'Game no 2020-Jan standard','Game no 2020-Jan rapid', 'Game no 2020-Jan blitz',
                                                             'Game no 2019-Dec standard','Game no 2019-Dec rapid', 'Game no 2019-Dec blitz',
                                                             'Game no 2019-Nov standard','Game no 2019-Nov rapid', 'Game no 2019-Nov blitz',
                                                             'Game no 2019-Oct standard','Game no 2019-Oct rapid', 'Game no 2019-Oct blitz',
                                                             'Game no 2019-Sep standard','Game no 2019-Sep rapid', 'Game no 2019-Sep blitz',
                                                             'Game no 2019-Aug standard','Game no 2019-Aug rapid', 'Game no 2019-Aug blitz',
                                                             'Game no 2019-Jul standard','Game no 2019-Jul rapid', 'Game no 2019-Jul blitz',
                                                             'Game no 2019-Jun standard','Game no 2019-Jun rapid', 'Game no 2019-Jun blitz',
                                                             'Game no 2019-May standard','Game no 2019-May rapid', 'Game no 2019-May blitz',
                                                             'Game no 2019-Apr standard','Game no 2019-Apr rapid', 'Game no 2019-Apr blitz',
                                                             'Game no 2019-Mar standard','Game no 2019-Mar rapid', 'Game no 2019-Mar blitz',
                                                             'Game no 2019-Feb standard','Game no 2019-Feb rapid', 'Game no 2019-Feb blitz',
                                                             'Game no 2019-Jan standard','Game no 2019-Jan rapid', 'Game no 2019-Jan blitz',                                                         
                                                             'Game no 2018-Dec standard','Game no 2018-Dec rapid', 'Game no 2018-Dec blitz',
                                                             'Game no 2018-Nov standard','Game no 2018-Nov rapid', 'Game no 2018-Nov blitz',
                                                             'Game no 2018-Oct standard','Game no 2018-Oct rapid', 'Game no 2018-Oct blitz',
                                                             'Game no 2018-Sep standard','Game no 2018-Sep rapid', 'Game no 2018-Sep blitz',
                                                             'Game no 2018-Aug standard','Game no 2018-Aug rapid', 'Game no 2018-Aug blitz',
                                                             'Game no 2018-Jul standard','Game no 2018-Jul rapid', 'Game no 2018-Jul blitz',
                                                             'Game no 2018-Jun standard','Game no 2018-Jun rapid', 'Game no 2018-Jun blitz',
                                                             'Game no 2018-May standard','Game no 2018-May rapid', 'Game no 2018-May blitz',
                                                             'Game no 2018-Apr standard','Game no 2018-Apr rapid', 'Game no 2018-Apr blitz',
                                                             'Game no 2018-Mar standard','Game no 2018-Mar rapid', 'Game no 2018-Mar blitz',
                                                             'Game no 2018-Feb standard','Game no 2018-Feb rapid', 'Game no 2018-Feb blitz',
                                                             'Game no 2018-Jan standard','Game no 2018-Jan rapid', 'Game no 2018-Jan blitz',                                                         
                                                             'Game no 2017-Dec standard','Game no 2017-Dec rapid', 'Game no 2017-Dec blitz',
                                                             'Game no 2017-Nov standard','Game no 2017-Nov rapid', 'Game no 2017-Nov blitz',
                                                             'Game no 2017-Oct standard','Game no 2017-Oct rapid', 'Game no 2017-Oct blitz',
                                                             'Game no 2017-Sep standard','Game no 2017-Sep rapid', 'Game no 2017-Sep blitz',
                                                             'Game no 2017-Aug standard','Game no 2017-Aug rapid', 'Game no 2017-Aug blitz',
                                                             'Game no 2017-Jul standard','Game no 2017-Jul rapid', 'Game no 2017-Jul blitz',
                                                             'Game no 2017-Jun standard','Game no 2017-Jun rapid', 'Game no 2017-Jun blitz',
                                                             'Game no 2017-May standard','Game no 2017-May rapid', 'Game no 2017-May blitz',
                                                             'Game no 2017-Apr standard','Game no 2017-Apr rapid', 'Game no 2017-Apr blitz',
                                                             'Game no 2017-Mar standard','Game no 2017-Mar rapid', 'Game no 2017-Mar blitz',
                                                             'Game no 2017-Feb standard','Game no 2017-Feb rapid', 'Game no 2017-Feb blitz',
                                                             'Game no 2017-Jan standard','Game no 2017-Jan rapid', 'Game no 2017-Jan blitz',                                                         
                                                             'Game no 2016-Dec standard','Game no 2016-Dec rapid', 'Game no 2016-Dec blitz',
                                                             'Game no 2016-Nov standard','Game no 2016-Nov rapid', 'Game no 2016-Nov blitz',
                                                             'Game no 2016-Oct standard','Game no 2016-Oct rapid', 'Game no 2016-Oct blitz',
                                                             'Game no 2016-Sep standard','Game no 2016-Sep rapid', 'Game no 2016-Sep blitz',
                                                             'Game no 2016-Aug standard','Game no 2016-Aug rapid', 'Game no 2016-Aug blitz',
                                                             'Game no 2016-Jul standard','Game no 2016-Jul rapid', 'Game no 2016-Jul blitz',
                                                             'Game no 2016-Jun standard','Game no 2016-Jun rapid', 'Game no 2016-Jun blitz',
                                                             'Game no 2016-May standard','Game no 2016-May rapid', 'Game no 2016-May blitz',
                                                             'Game no 2016-Apr standard','Game no 2016-Apr rapid', 'Game no 2016-Apr blitz',
                                                             'Game no 2016-Mar standard','Game no 2016-Mar rapid', 'Game no 2016-Mar blitz',
                                                             'Game no 2016-Feb standard','Game no 2016-Feb rapid', 'Game no 2016-Feb blitz',
                                                             'Game no 2016-Jan standard','Game no 2016-Jan rapid', 'Game no 2016-Jan blitz',                                                         
                                                             'Game no 2015-Dec standard','Game no 2015-Dec rapid', 'Game no 2015-Dec blitz',
                                                             'Game no 2015-Nov standard','Game no 2015-Nov rapid', 'Game no 2015-Nov blitz',
                                                             'Game no 2015-Oct standard','Game no 2015-Oct rapid', 'Game no 2015-Oct blitz',
                                                             'Game no 2015-Sep standard','Game no 2015-Sep rapid', 'Game no 2015-Sep blitz',
                                                             'Game no 2015-Aug standard','Game no 2015-Aug rapid', 'Game no 2015-Aug blitz',
                                                             'Game no 2015-Jul standard','Game no 2015-Jul rapid', 'Game no 2015-Jul blitz',
                                                             'Game no 2015-Jun standard','Game no 2015-Jun rapid', 'Game no 2015-Jun blitz',
                                                             'Game no 2015-May standard','Game no 2015-May rapid', 'Game no 2015-May blitz',
                                                             'Game no 2015-Apr standard','Game no 2015-Apr rapid', 'Game no 2015-Apr blitz',
                                                             'Game no 2015-Mar standard','Game no 2015-Mar rapid', 'Game no 2015-Mar blitz',
                                                             'Game no 2015-Feb standard','Game no 2015-Feb rapid', 'Game no 2015-Feb blitz',                                                         
                                                             'Rating change 2021-Feb standard','Rating change 2021-Feb rapid', 'Rating change 2021-Feb blitz',
                                                             'Rating change 2021-Jan standard','Rating change 2021-Jan rapid', 'Rating change 2021-Jan blitz',
                                                             'Rating change 2020-Dec standard','Rating change 2020-Dec rapid', 'Rating change 2020-Dec blitz',
                                                             'Rating change 2020-Nov standard','Rating change 2020-Nov rapid', 'Rating change 2020-Nov blitz',
                                                             'Rating change 2020-Oct standard','Rating change 2020-Oct rapid', 'Rating change 2020-Oct blitz',
                                                             'Rating change 2020-Sep standard','Rating change 2020-Sep rapid', 'Rating change 2020-Sep blitz',
                                                             'Rating change 2020-Aug standard','Rating change 2020-Aug rapid', 'Rating change 2020-Aug blitz',
                                                             'Rating change 2020-Jul standard','Rating change 2020-Jul rapid', 'Rating change 2020-Jul blitz',
                                                             'Rating change 2020-Jun standard','Rating change 2020-Jun rapid', 'Rating change 2020-Jun blitz',
                                                             'Rating change 2020-May standard','Rating change 2020-May rapid', 'Rating change 2020-May blitz',
                                                             'Rating change 2020-Apr standard','Rating change 2020-Apr rapid', 'Rating change 2020-Apr blitz',
                                                             'Rating change 2020-Mar standard','Rating change 2020-Mar rapid', 'Rating change 2020-Mar blitz',
                                                             'Rating change 2020-Feb standard','Rating change 2020-Feb rapid', 'Rating change 2020-Feb blitz',
                                                             'Rating change 2020-Jan standard','Rating change 2020-Jan rapid', 'Rating change 2020-Jan blitz',
                                                             'Rating change 2019-Dec standard','Rating change 2019-Dec rapid', 'Rating change 2019-Dec blitz',
                                                             'Rating change 2019-Nov standard','Rating change 2019-Nov rapid', 'Rating change 2019-Nov blitz',
                                                             'Rating change 2019-Oct standard','Rating change 2019-Oct rapid', 'Rating change 2019-Oct blitz',
                                                             'Rating change 2019-Sep standard','Rating change 2019-Sep rapid', 'Rating change 2019-Sep blitz',
                                                             'Rating change 2019-Aug standard','Rating change 2019-Aug rapid', 'Rating change 2019-Aug blitz',
                                                             'Rating change 2019-Jul standard','Rating change 2019-Jul rapid', 'Rating change 2019-Jul blitz',
                                                             'Rating change 2019-Jun standard','Rating change 2019-Jun rapid', 'Rating change 2019-Jun blitz',
                                                             'Rating change 2019-May standard','Rating change 2019-May rapid', 'Rating change 2019-May blitz',
                                                             'Rating change 2019-Apr standard','Rating change 2019-Apr rapid', 'Rating change 2019-Apr blitz',
                                                             'Rating change 2019-Mar standard','Rating change 2019-Mar rapid', 'Rating change 2019-Mar blitz',
                                                             'Rating change 2019-Feb standard','Rating change 2019-Feb rapid', 'Rating change 2019-Feb blitz',
                                                             'Rating change 2019-Jan standard','Rating change 2019-Jan rapid', 'Rating change 2019-Jan blitz',
                                                             'Rating change 2018-Dec standard','Rating change 2018-Dec rapid', 'Rating change 2018-Dec blitz',
                                                             'Rating change 2018-Nov standard','Rating change 2018-Nov rapid', 'Rating change 2018-Nov blitz',
                                                             'Rating change 2018-Oct standard','Rating change 2018-Oct rapid', 'Rating change 2018-Oct blitz',
                                                             'Rating change 2018-Sep standard','Rating change 2018-Sep rapid', 'Rating change 2018-Sep blitz',
                                                             'Rating change 2018-Aug standard','Rating change 2018-Aug rapid', 'Rating change 2018-Aug blitz',
                                                             'Rating change 2018-Jul standard','Rating change 2018-Jul rapid', 'Rating change 2018-Jul blitz',
                                                             'Rating change 2018-Jun standard','Rating change 2018-Jun rapid', 'Rating change 2018-Jun blitz',
                                                             'Rating change 2018-May standard','Rating change 2018-May rapid', 'Rating change 2018-May blitz',
                                                             'Rating change 2018-Apr standard','Rating change 2018-Apr rapid', 'Rating change 2018-Apr blitz',
                                                             'Rating change 2018-Mar standard','Rating change 2018-Mar rapid', 'Rating change 2018-Mar blitz',
                                                             'Rating change 2018-Feb standard','Rating change 2018-Feb rapid', 'Rating change 2018-Feb blitz',
                                                             'Rating change 2018-Jan standard','Rating change 2018-Jan rapid', 'Rating change 2018-Jan blitz',                                                         
                                                             'Rating change 2017-Dec standard','Rating change 2017-Dec rapid', 'Rating change 2017-Dec blitz',
                                                             'Rating change 2017-Nov standard','Rating change 2017-Nov rapid', 'Rating change 2017-Nov blitz',
                                                             'Rating change 2017-Oct standard','Rating change 2017-Oct rapid', 'Rating change 2017-Oct blitz',
                                                             'Rating change 2017-Sep standard','Rating change 2017-Sep rapid', 'Rating change 2017-Sep blitz',
                                                             'Rating change 2017-Aug standard','Rating change 2017-Aug rapid', 'Rating change 2017-Aug blitz',
                                                             'Rating change 2017-Jul standard','Rating change 2017-Jul rapid', 'Rating change 2017-Jul blitz',
                                                             'Rating change 2017-Jun standard','Rating change 2017-Jun rapid', 'Rating change 2017-Jun blitz',
                                                             'Rating change 2017-May standard','Rating change 2017-May rapid', 'Rating change 2017-May blitz',
                                                             'Rating change 2017-Apr standard','Rating change 2017-Apr rapid', 'Rating change 2017-Apr blitz',
                                                             'Rating change 2017-Mar standard','Rating change 2017-Mar rapid', 'Rating change 2017-Mar blitz',
                                                             'Rating change 2017-Feb standard','Rating change 2017-Feb rapid', 'Rating change 2017-Feb blitz',
                                                             'Rating change 2017-Jan standard','Rating change 2017-Jan rapid', 'Rating change 2017-Jan blitz',                                                         
                                                             'Rating change 2016-Dec standard','Rating change 2016-Dec rapid', 'Rating change 2016-Dec blitz',
                                                             'Rating change 2016-Nov standard','Rating change 2016-Nov rapid', 'Rating change 2016-Nov blitz',
                                                             'Rating change 2016-Oct standard','Rating change 2016-Oct rapid', 'Rating change 2016-Oct blitz',
                                                             'Rating change 2016-Sep standard','Rating change 2016-Sep rapid', 'Rating change 2016-Sep blitz',
                                                             'Rating change 2016-Aug standard','Rating change 2016-Aug rapid', 'Rating change 2016-Aug blitz',
                                                             'Rating change 2016-Jul standard','Rating change 2016-Jul rapid', 'Rating change 2016-Jul blitz',
                                                             'Rating change 2016-Jun standard','Rating change 2016-Jun rapid', 'Rating change 2016-Jun blitz',
                                                             'Rating change 2016-May standard','Rating change 2016-May rapid', 'Rating change 2016-May blitz',
                                                             'Rating change 2016-Apr standard','Rating change 2016-Apr rapid', 'Rating change 2016-Apr blitz',
                                                             'Rating change 2016-Mar standard','Rating change 2016-Mar rapid', 'Rating change 2016-Mar blitz',
                                                             'Rating change 2016-Feb standard','Rating change 2016-Feb rapid', 'Rating change 2016-Feb blitz',
                                                             'Rating change 2016-Jan standard','Rating change 2016-Jan rapid', 'Rating change 2016-Jan blitz',                                                         
                                                             'Rating change 2015-Dec standard','Rating change 2015-Dec rapid', 'Rating change 2015-Dec blitz',
                                                             'Rating change 2015-Nov standard','Rating change 2015-Nov rapid', 'Rating change 2015-Nov blitz',
                                                             'Rating change 2015-Oct standard','Rating change 2015-Oct rapid', 'Rating change 2015-Oct blitz',
                                                             'Rating change 2015-Sep standard','Rating change 2015-Sep rapid', 'Rating change 2015-Sep blitz',
                                                             'Rating change 2015-Aug standard','Rating change 2015-Aug rapid', 'Rating change 2015-Aug blitz',
                                                             'Rating change 2015-Jul standard','Rating change 2015-Jul rapid', 'Rating change 2015-Jul blitz',
                                                             'Rating change 2015-Jun standard','Rating change 2015-Jun rapid', 'Rating change 2015-Jun blitz',
                                                             'Rating change 2015-May standard','Rating change 2015-May rapid', 'Rating change 2015-May blitz',
                                                             'Rating change 2015-Apr standard','Rating change 2015-Apr rapid', 'Rating change 2015-Apr blitz',
                                                             'Rating change 2015-Mar standard','Rating change 2015-Mar rapid', 'Rating change 2015-Mar blitz',
                                                             'Rating change 2015-Feb standard','Rating change 2015-Feb rapid', 'Rating change 2015-Feb blitz',
                                                             'Rating change 2015-Jan standard','Rating change 2015-Jan rapid', 'Rating change 2015-Jan blitz']
        

        
        if(num_col == 463):
            full_columns.remove('Round 10')
            full_columns.remove('Round 11')
            
        elif(num_col == 464):
            full_columns.remove('Round 11')
            
            
        test = pd.read_csv(filename, encoding='utf8',index_col=0, names=full_columns)
        test['Season'] = season
        test['League'] = league
        
        if(num_col == 463):
            test['Round 10'] = 'No game was palyed'
            test['Round 11'] = 'No game was palyed'
            
        elif(num_col == 464):
            test['Round 11'] = 'No game was palyed'
        
        
        
        

        dfs.append(test)
 
    all_games = pd.concat(dfs, ignore_index=True)
    return all_games
    
    
    
    


all_ranklist=load_rankingLists()
all_ranklist
old_ranklist = all_ranklist


In [7]:
all_ranklist[all_ranklist['Player name'] == 'Szukics Ádám']['Born year']
# i have to drop born_year because of invalid data
del all_ranklist['Born year']
all_ranklist.reindex(all_ranklist.columns, axis=1)

,Player name,Youth,Female,Nationality,Title,Elo,Round 1,Round 2,Round 3,Round 4,...,Rating change 2015-Feb standard,Rating change 2015-Feb rapid,Rating change 2015-Feb blitz,Rating change 2015-Jan standard,Rating change 2015-Jan rapid,Rating change 2015-Jan blitz,Season,League,Round 10,Round 11
0,Anand Viswanathan,NaN,NaN,IND,GM,2816.0,NaN,NaN,NaN,NaN,...,No Games,No Games,No Games,Available 3.9,Available -1,Available -44,15-16,nb1,No game was palyed,No game was palyed
1,Naiditsch Arkadij,NaN,NaN,GER,GM,2684.0,NaN,NaN,NaN,NaN,...,Available 12.4,No Games,No Games,Available -37.1,No Games,No Games,15-16,nb1,No game was palyed,No game was palyed
2,Shirov Alexei,NaN,NaN,ESP,GM,2712.0,NaN,NaN,NaN,NaN,...,No Games,No Games,No Games,Available 14.2,Available -30.4,No Games,15-16,nb1,No game was palyed,No game was palyed
3,Efimenko Zahar,NaN,NaN,UKR,GM,2647.0,NaN,NaN,NaN,NaN,...,Available 0.8,No Games,No Games,Available 11.3,Available -20.6,Available 23.6,15-16,nb1,No game was palyed,No game was palyed
4,Balogh Csaba,NaN,NaN,HUN,GM,2659.0,NaN,NaN,½,NaN,...,Available -0.1,No Games,No Games,Available 1.1,No Games,No Games,15-16,nb1,No game was palyed,No game was palyed
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20452,Horváth Dominik,Ifi,NaN,HUN,NaN,1335.0,½,1,NaN,0,...,NaN,NaN,NaN,NaN,NaN,NaN,19-20,nb2toth,No game was palyed,No game was palyed
20453,Gottfried Zsolt,NaN,NaN,HUN,NaN,1279.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,19-20,nb2toth,No game was palyed,No game was palyed
20454,Kocmar Kitti,Ifi,Nõ,HUN,NaN,1325.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,19-20,nb2toth,No game was palyed,No game was palyed
20455,Csesznok Balázs István,Ifi,NaN,HUN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,19-20,nb2toth,No game was palyed,No game was palyed


In [8]:
%%time

import datetime

def manipulate_ranklist(df):
    

    df.replace(r'\s+',np.nan,regex=True).replace('',np.nan)
#     df = df.rename(columns={'Born year': 'Age'})

#     df['Age'] = df['Age'].fillna(1980)       #usually people with only national chess identifiers
#     df['Age'] = df['Age'].apply(np.int64)
#     df['Age'] = df['Age'].apply(lambda x: datetime.datetime.now().year-x ) # 2021 as of now 
    
#     age_season_dict = {'19-20' : 1 , '18-19' : 2, '17-18' : 3, '16-17' : 4 , '15-16' : 5 }
    
#     df['Age'] = df.apply(lambda x: x['Age']-age_season_dict.get(x['Season']), axis=1) # go backwards in seasons

    df['Youth'] = df['Youth'].apply(lambda x: 1 if x == 'Ifi' else 0)
    #df['Youth'] = df['Youth'].apply(np.float64)

    df['Female'] = df['Female'].apply(lambda x: 1 if x == 'Nõ' else 0)
    df['Female'] = df['Female'].fillna(0)
    #df['Female'] = df['Female'].astype(int)
    
    df['Elo'] = df['Elo'].fillna(1300)
    #df['Elo'] = df['Elo'].apply(np.float64)
    
    
    df['Standard elo'] = df['Standard elo'].fillna(1300)
    #df['Standard elo'] = df['Standard elo'].apply(np.float64)
    
    df['Rapid elo'] = df['Rapid elo'].fillna(1300)
    #df['Rapid elo'] = df['Rapid elo'].apply(np.float64)
    
    df['Blitz elo'] = df['Blitz elo'].apply(lambda x: x if not isinstance(x, str) else np.nan)
    df['Blitz elo'] = df['Blitz elo'].fillna(1300)
    #df['Blitz elo'] = df['Blitz elo'].apply(np.float64)

    df['Total'] = df['Total'].fillna('0/0')
    df['Total'] = df['Total'].apply(lambda x: x if x[0].isdigit() else '0/0') #sometimes the value here is the team name, so i check for the first character
    
    
    df[['Total points', 'Total games']] = df['Total'].str.split('/', n=1, expand=True)
    #df['Total points'] = df['Total points'].astype(float)
    #df['Total games'] = df['Total games'].astype(int)
    
    del df['Total']
    correct_df = df.apply(pd.to_numeric, errors='ignore')  #converts whatever it can to numeric data type 
    
    
    correct_df.reindex(df.columns, axis=1)
    
    
    
    return correct_df
    
all_ranklist = manipulate_ranklist(all_ranklist)
all_ranklist

Wall time: 33 s


,Player name,Youth,Female,Nationality,Title,Elo,Round 1,Round 2,Round 3,Round 4,...,Rating change 2015-Feb blitz,Rating change 2015-Jan standard,Rating change 2015-Jan rapid,Rating change 2015-Jan blitz,Season,League,Round 10,Round 11,Total points,Total games
0,Anand Viswanathan,0,0,IND,GM,2816.0,NaN,NaN,NaN,NaN,...,No Games,Available 3.9,Available -1,Available -44,15-16,nb1,No game was palyed,No game was palyed,0.0,0
1,Naiditsch Arkadij,0,0,GER,GM,2684.0,NaN,NaN,NaN,NaN,...,No Games,Available -37.1,No Games,No Games,15-16,nb1,No game was palyed,No game was palyed,1.0,2
2,Shirov Alexei,0,0,ESP,GM,2712.0,NaN,NaN,NaN,NaN,...,No Games,Available 14.2,Available -30.4,No Games,15-16,nb1,No game was palyed,No game was palyed,0.0,0
3,Efimenko Zahar,0,0,UKR,GM,2647.0,NaN,NaN,NaN,NaN,...,No Games,Available 11.3,Available -20.6,Available 23.6,15-16,nb1,No game was palyed,No game was palyed,1.0,1
4,Balogh Csaba,0,0,HUN,GM,2659.0,NaN,NaN,½,NaN,...,No Games,Available 1.1,No Games,No Games,15-16,nb1,No game was palyed,No game was palyed,3.5,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20452,Horváth Dominik,1,0,HUN,NaN,1335.0,½,1,NaN,0,...,NaN,NaN,NaN,NaN,19-20,nb2toth,No game was palyed,No game was palyed,1.5,5
20453,Gottfried Zsolt,0,0,HUN,NaN,1279.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,19-20,nb2toth,No game was palyed,No game was palyed,0.0,0
20454,Kocmar Kitti,1,1,HUN,NaN,1325.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,19-20,nb2toth,No game was palyed,No game was palyed,0.0,0
20455,Csesznok Balázs István,1,0,HUN,NaN,1300.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,19-20,nb2toth,No game was palyed,No game was palyed,0.0,0


In [9]:
all_ranklist['Female'].dtype

dtype('int64')

In [10]:
#all_ranklist[all_ranklist['Age'] < 0 ]['Age']

In [11]:
%%time

def get_column_index(df, column_name):
    return df.columns.get_loc(column_name)


def get_start_end_column_index(df,start_column_name, end_column_name):
    start = get_column_index(df, start_column_name)
    end = get_column_index(df, end_column_name)
    
    return start, end

def clean_fide_stats(df1):
    
    df = df1.copy()
    
    start, end = get_start_end_column_index(df, 'Game no 2021-Feb standard', 'Rating change 2015-Jan blitz')
    
    for i in range(start,end+1):
        df.iloc[:, i] = df.iloc[:, i].fillna('0')
        df.iloc[:, i] = df.iloc[:, i].str.extract('(-?\d+)', expand=False)
        df.iloc[:, i] = df.iloc[:, i].fillna('0')
        df.iloc[:, i] = df.iloc[:, i].astype(int)
        
    return df

def sum_fide(df, year, column_type, game_type):
    
    if(year == 2015 and column_type=='Game no'):
        return df.iloc[:, get_column_index(df, column_type+' '+str(year)+'-Dec '+game_type):get_column_index(df, column_type+' '+str(year)+'-Feb '+game_type)+1:3].sum(axis=1)
        #print(df.iloc[:, get_column_index(df, column_type+' '+str(year)+'-Dec '+game_type):get_column_index(df, column_type+' '+str(year)+'-Feb '+game_type)+1:3].sum(axis=1))
    else:
        return df.iloc[:, get_column_index(df, column_type+' '+str(year)+'-Dec '+game_type):get_column_index(df, column_type+' '+str(year)+'-Jan '+game_type)+1:3].sum(axis=1)
        #print(df.iloc[:, get_column_index(df, column_type+' '+str(year)+'-Dec '+game_type):get_column_index(df, column_type+' '+str(year)+'-Jan '+game_type)+1:3].sum(axis=1))


def summarize_fide_columns(df1):
    #total = df.iloc[:, get_column_index(df, 'Game no 2021-Feb standard'):get_column_index(df, 'Game no 2020-Jan standard'):3].sum(axis=1)
    #Game no 2021-Feb standard_home
    #print(total.max())
    #TODO: minden évre partik száma + élő változás kategóriánként (standard,rapid,blitz)
    df = df1.copy()
    for i in range(0,6):  # ONLY GOES UP TO 2020, 2021 IS NOT CALCULATED!

        df[str(2015+i)+' Year standard games total'] = sum_fide(df1, 2015+i, 'Game no', 'standard')#.apply(np.float64)
        df[str(2015+i)+' Year rapid games total'] = sum_fide(df1, 2015+i, 'Game no', 'rapid')#.apply(np.float64)
        df[str(2015+i)+' Year blitz games total'] = sum_fide(df1, 2015+i, 'Game no', 'blitz')#.apply(np.float64)

        
        df[str(2015+i)+' Year standard rating change total'] = sum_fide(df1, 2015+i, 'Rating change', 'standard')#.apply(np.float64)
        df[str(2015+i)+' Year rapid rating change total'] = sum_fide(df1, 2015+i, 'Rating change', 'rapid')#.apply(np.float64)
        df[str(2015+i)+' Year blitz rating change total'] = sum_fide(df1, 2015+i, 'Rating change', 'blitz')#.apply(np.float64)

        
    df.drop(df.iloc[:, get_column_index(df, 'Game no 2021-Feb standard'): get_column_index(df,'Rating change 2015-Jan blitz')+1], inplace = True, axis = 1) #+1 so that the last one is included as well
    df = df.apply(pd.to_numeric, errors='ignore')
    df.reindex(df.columns, axis=1)
    
    return df


all_ranklist = clean_fide_stats(all_ranklist)

all_ranklist.describe()

all_ranklist = summarize_fide_columns(all_ranklist)

all_ranklist.dtypes

Wall time: 1min 56s


Player name                               object
Youth                                      int64
Female                                     int64
Nationality                               object
Title                                     object
                                           ...  
2020 Year rapid games total                int64
2020 Year blitz games total                int64
2020 Year standard rating change total     int64
2020 Year rapid rating change total        int64
2020 Year blitz rating change total        int64
Length: 61, dtype: object

In [12]:
#all_ranklist = summarize_fide_columns(all_ranklist)
all_ranklist['Rapid elo'] = all_ranklist['Rapid elo'].astype(float)
#all_ranklist.select_dtypes(include=[np.object]).dtypes
all_ranklist['Rapid elo'].dtype

dtype('float64')

In [13]:
df = pd.merge(df.astype(str), all_ranklist.add_suffix('_home').astype(str), left_on=['Player home', 'Home team', 'Season', 'League'], right_on=['Player name_home', 'Team name_home','Season_home', 'League_home'], how='left')
df = df[df['Season'] == df['Season_home']]  #had some rows where season column contained NaN values

df  # 26788 rows × 80 columns


,Home team,Away team,Board no.,Title home,Elo home,Player home,Color home,Color away,Player away,Elo away,...,2019 Year blitz games total_home,2019 Year standard rating change total_home,2019 Year rapid rating change total_home,2019 Year blitz rating change total_home,2020 Year standard games total_home,2020 Year rapid games total_home,2020 Year blitz games total_home,2020 Year standard rating change total_home,2020 Year rapid rating change total_home,2020 Year blitz rating change total_home
0,HÜSI SC,HVSE-Infraplan,1,IM,2374,Böröcz István,1,0,Jankovic Alojzije,2557,...,0,-32,0,0,26,0,0,-1,0,0
1,HÜSI SC,HVSE-Infraplan,2,No Title,2362,Havasi Gergő,0,1,Ruck Róbert,2565,...,0,0,0,0,4,0,0,0,0,0
2,HÜSI SC,HVSE-Infraplan,3,GM,2470,Kosic Dragan,1,0,Németh Miklós,2469,...,0,-16,0,0,80,9,0,-27,-37,0
3,HÜSI SC,HVSE-Infraplan,4,IM,2424,Stankovic Milos,0,1,Dr. Kovács Gábor,2462,...,35,16,14,35,63,11,13,-24,-18,-29
4,HÜSI SC,HVSE-Infraplan,5,GM,2426,Drazic Sinisa,1,0,Mészáros Michal,2389,...,178,-42,60,122,128,59,254,-53,-74,-84
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28174,Félegyháza-Csongrád,Gyulai SE,8,No Title,1924,Kelemen Sándor,0,1,Oltean Cristian-Daniel,1911,...,0,-24,0,0,5,0,0,0,0,0
28175,Félegyháza-Csongrád,Gyulai SE,9,No Title,1943,Szekeres Sebestyén,1,0,Györgydeák Tibor,1766,...,0,-24,0,0,3,0,0,5,0,0
28176,Félegyháza-Csongrád,Gyulai SE,10,No Title,1738,Hideg Ferenc Bendegúz,0,1,Gurzó Bence Bendeguz,1571,...,9,-34,55,19,12,0,0,-10,0,0
28177,Félegyháza-Csongrád,Gyulai SE,11,No Title,1590,Nagy Noémi,1,0,Vaszkó Szabolcs,1400,...,9,22,35,45,9,0,11,39,0,-23


In [14]:
df = pd.merge(df, all_ranklist.add_suffix('_away'), left_on=['Player away', 'Away team', 'Season', 'League'], right_on=['Player name_away', 'Team name_away', 'Season_away', 'League_away'], how='left')
df = df[df['Season'] == df['Season_away']]
df.drop_duplicates(keep=False, inplace=True)

In [15]:
df.columns.values

array(['Home team', 'Away team', 'Board no. ', 'Title home', 'Elo home',
       'Player home', 'Color home', 'Color away', 'Player away',
       'Elo away', 'Title away', 'Date', 'Time', 'League', 'Round',
       'Season', 'Result_home', 'Result_away', 'Player name_home',
       'Youth_home', 'Female_home', 'Nationality_home', 'Title_home',
       'Elo_home', 'Round 1_home', 'Round 2_home', 'Round 3_home',
       'Round 4_home', 'Round 5_home', 'Round 6_home', 'Round 7_home',
       'Round 8_home', 'Round 9_home', 'Team name_home',
       'Standard elo_home', 'Rapid elo_home', 'Blitz elo_home',
       'Season_home', 'League_home', 'Round 10_home', 'Round 11_home',
       'Total points_home', 'Total games_home',
       '2015 Year standard games total_home',
       '2015 Year rapid games total_home',
       '2015 Year blitz games total_home',
       '2015 Year standard rating change total_home',
       '2015 Year rapid rating change total_home',
       '2015 Year blitz rating change tota

In [16]:
df

,Home team,Away team,Board no.,Title home,Elo home,Player home,Color home,Color away,Player away,Elo away,...,2019 Year blitz games total_away,2019 Year standard rating change total_away,2019 Year rapid rating change total_away,2019 Year blitz rating change total_away,2020 Year standard games total_away,2020 Year rapid games total_away,2020 Year blitz games total_away,2020 Year standard rating change total_away,2020 Year rapid rating change total_away,2020 Year blitz rating change total_away
0,HÜSI SC,HVSE-Infraplan,1,IM,2374,Böröcz István,1,0,Jankovic Alojzije,2557,...,70.0,-16.0,-46.0,16.0,2.0,9.0,23.0,1.0,20.0,3.0
1,HÜSI SC,HVSE-Infraplan,2,No Title,2362,Havasi Gergő,0,1,Ruck Róbert,2565,...,66.0,17.0,3.0,-52.0,17.0,41.0,67.0,15.0,-85.0,27.0
2,HÜSI SC,HVSE-Infraplan,3,GM,2470,Kosic Dragan,1,0,Németh Miklós,2469,...,16.0,13.0,0.0,-1.0,26.0,8.0,0.0,-8.0,-6.0,0.0
3,HÜSI SC,HVSE-Infraplan,4,IM,2424,Stankovic Milos,0,1,Dr. Kovács Gábor,2462,...,11.0,1.0,0.0,-51.0,7.0,0.0,0.0,0.0,0.0,0.0
4,HÜSI SC,HVSE-Infraplan,5,GM,2426,Drazic Sinisa,1,0,Mészáros Michal,2389,...,0.0,9.0,0.0,0.0,2.0,0.0,0.0,-3.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27062,Félegyháza-Csongrád,Gyulai SE,8,No Title,1924,Kelemen Sándor,0,1,Oltean Cristian-Daniel,1911,...,33.0,147.0,138.0,312.0,13.0,37.0,8.0,195.0,126.0,81.0
27063,Félegyháza-Csongrád,Gyulai SE,9,No Title,1943,Szekeres Sebestyén,1,0,Györgydeák Tibor,1766,...,0.0,26.0,0.0,0.0,9.0,0.0,0.0,-4.0,0.0,0.0
27064,Félegyháza-Csongrád,Gyulai SE,10,No Title,1738,Hideg Ferenc Bendegúz,0,1,Gurzó Bence Bendeguz,1571,...,0.0,7.0,0.0,0.0,5.0,0.0,0.0,34.0,0.0,0.0
27065,Félegyháza-Csongrád,Gyulai SE,11,No Title,1590,Nagy Noémi,1,0,Vaszkó Szabolcs,1400,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [17]:
df.describe()

,Youth_away,Female_away,Elo_away,Standard elo_away,Rapid elo_away,Blitz elo_away,Total points_away,Total games_away,2015 Year standard games total_away,2015 Year rapid games total_away,...,2019 Year blitz games total_away,2019 Year standard rating change total_away,2019 Year rapid rating change total_away,2019 Year blitz rating change total_away,2020 Year standard games total_away,2020 Year rapid games total_away,2020 Year blitz games total_away,2020 Year standard rating change total_away,2020 Year rapid rating change total_away,2020 Year blitz rating change total_away
count,26353.00000,26353.000000,26353.000000,26353.000000,26353.000000,26353.000000,26353.000000,26353.000000,26353.000000,26353.000000,...,26353.000000,26353.000000,26353.000000,26353.000000,26353.000000,26353.000000,26353.000000,26353.000000,26353.000000,26353.000000
mean,0.19641,0.056616,2015.209350,2016.978788,1711.709445,1642.010473,3.846830,7.587106,26.016583,4.155087,...,3.209767,-0.194361,-0.104049,0.366068,12.365044,2.802110,3.203696,1.187379,-0.570827,-0.179069
std,0.39729,0.231112,280.694185,269.145420,413.576083,425.904811,2.103847,2.850693,29.405721,11.462935,...,11.262331,43.051906,29.030987,18.693289,15.668418,7.134068,10.445573,35.585177,24.564858,21.865165
min,0.00000,0.000000,1004.000000,1024.000000,1049.000000,1068.000000,0.000000,0.000000,-160.000000,-68.000000,...,-22.000000,-258.000000,-190.000000,-148.000000,0.000000,0.000000,0.000000,-211.000000,-161.000000,-166.000000
25%,0.00000,0.000000,1866.000000,1867.000000,1300.000000,1300.000000,2.500000,6.000000,7.000000,0.000000,...,0.000000,-20.000000,0.000000,0.000000,3.000000,0.000000,0.000000,-11.000000,0.000000,0.000000
50%,0.00000,0.000000,2038.000000,2027.000000,1716.000000,1300.000000,4.000000,8.000000,16.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,7.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,0.00000,0.000000,2200.000000,2197.000000,2065.000000,2052.000000,5.500000,10.000000,37.000000,7.000000,...,0.000000,12.000000,0.000000,0.000000,16.000000,0.000000,0.000000,7.000000,0.000000,0.000000
max,1.00000,1.000000,2752.000000,2763.000000,2727.000000,2759.000000,11.000000,11.000000,300.000000,262.000000,...,178.000000,453.000000,310.000000,312.000000,144.000000,99.000000,254.000000,487.000000,263.000000,245.000000


In [18]:
#!pip install pandas_profiling

#import pandas_profiling

#profile = pandas_profiling.ProfileReport(df, minimal=True)
#profile.to_notebook_iframe()

**TODO:**
- elo-nel vannak kiugrok: 2900 feletti elos sorokat droppold -> DONE
- 18-19es szezonban 2005 partim van ?!M!M                    -> DONE
- age visszavezetve szezonra                                 -> DONE
- leageu home vs league away
- 2015Year standard games total_home negativ ertek??
- title-away missing values
- round1-2-3... missing values

In [19]:
df.iloc[:,:20]

,Home team,Away team,Board no.,Title home,Elo home,Player home,Color home,Color away,Player away,Elo away,Title away,Date,Time,League,Round,Season,Result_home,Result_away,Player name_home,Youth_home
0,HÜSI SC,HVSE-Infraplan,1,IM,2374,Böröcz István,1,0,Jankovic Alojzije,2557,GM,2015-11-07,15:00,nb1,0,15-16,0.0,1.0,Böröcz István,0
1,HÜSI SC,HVSE-Infraplan,2,No Title,2362,Havasi Gergő,0,1,Ruck Róbert,2565,GM,2015-11-07,15:00,nb1,0,15-16,0.5,0.5,Havasi Gergő,1
2,HÜSI SC,HVSE-Infraplan,3,GM,2470,Kosic Dragan,1,0,Németh Miklós,2469,IM,2015-11-07,15:00,nb1,0,15-16,0.5,0.5,Kosic Dragan,0
3,HÜSI SC,HVSE-Infraplan,4,IM,2424,Stankovic Milos,0,1,Dr. Kovács Gábor,2462,IM,2015-11-07,15:00,nb1,0,15-16,0.5,0.5,Stankovic Milos,0
4,HÜSI SC,HVSE-Infraplan,5,GM,2426,Drazic Sinisa,1,0,Mészáros Michal,2389,IM,2015-11-07,15:00,nb1,0,15-16,0.5,0.5,Drazic Sinisa,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27062,Félegyháza-Csongrád,Gyulai SE,8,No Title,1924,Kelemen Sándor,0,1,Oltean Cristian-Daniel,1911,No Title,2020-03-08,10:00,nb2toth,6,19-20,0.0,1.0,Kelemen Sándor,0
27063,Félegyháza-Csongrád,Gyulai SE,9,No Title,1943,Szekeres Sebestyén,1,0,Györgydeák Tibor,1766,No Title,2020-03-08,10:00,nb2toth,6,19-20,0.5,0.5,Szekeres Sebestyén,0
27064,Félegyháza-Csongrád,Gyulai SE,10,No Title,1738,Hideg Ferenc Bendegúz,0,1,Gurzó Bence Bendeguz,1571,No Title,2020-03-08,10:00,nb2toth,6,19-20,0.5,0.5,Hideg Ferenc Bendegúz,0
27065,Félegyháza-Csongrád,Gyulai SE,11,No Title,1590,Nagy Noémi,1,0,Vaszkó Szabolcs,1400,-,2020-03-08,10:00,nb2toth,6,19-20,1.0,0.0,Nagy Noémi,0


In [20]:
df.select_dtypes(include=[np.object]).dtypes

Home team         object
Away team         object
Board no.         object
Title home        object
Elo home          object
                   ...  
Team name_away    object
Season_away       object
League_away       object
Round 10_away     object
Round 11_away     object
Length: 96, dtype: object

In [21]:
# my previous EDA showed that some elo ratings are invalid

df['Elo home'] = df['Elo home'].astype('int64')
df['Elo away'] = df['Elo away'].astype('int64')

df.drop(df[df['Elo home'] > 2800].index, inplace=True)
df.drop(df[df['Elo away'] > 2800].index, inplace=True)
df.shape

(26179, 140)

In [22]:
import seaborn as sns
import matplotlib.pyplot as plt

# df_num = df.select_dtypes(include=[np.number])
# df_num.info()

# plt.figure(figsize = (20,16))
# sns.heatmap(df_num.sample(1000), annot=False, xticklabels=True, yticklabels=True)

df.columns.values

array(['Home team', 'Away team', 'Board no. ', 'Title home', 'Elo home',
       'Player home', 'Color home', 'Color away', 'Player away',
       'Elo away', 'Title away', 'Date', 'Time', 'League', 'Round',
       'Season', 'Result_home', 'Result_away', 'Player name_home',
       'Youth_home', 'Female_home', 'Nationality_home', 'Title_home',
       'Elo_home', 'Round 1_home', 'Round 2_home', 'Round 3_home',
       'Round 4_home', 'Round 5_home', 'Round 6_home', 'Round 7_home',
       'Round 8_home', 'Round 9_home', 'Team name_home',
       'Standard elo_home', 'Rapid elo_home', 'Blitz elo_home',
       'Season_home', 'League_home', 'Round 10_home', 'Round 11_home',
       'Total points_home', 'Total games_home',
       '2015 Year standard games total_home',
       '2015 Year rapid games total_home',
       '2015 Year blitz games total_home',
       '2015 Year standard rating change total_home',
       '2015 Year rapid rating change total_home',
       '2015 Year blitz rating change tota

In [23]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 26179 entries, 0 to 27066
Columns: 140 entries, Home team to 2020 Year blitz rating change total_away
dtypes: float64(44), int64(2), object(94)
memory usage: 28.2+ MB


In [24]:
test = df.copy()
test = test.select_dtypes(['object']).apply(pd.Series.astype, dtype='category', axis=1).reindex(test.columns, axis=1)

test = pd.get_dummies(df, columns = ['Title home', 'Title away']) 

test

,Home team,Away team,Board no.,Elo home,Player home,Color home,Color away,Player away,Elo away,Date,...,Title away_-,Title away_CM,Title away_FM,Title away_GM,Title away_IM,Title away_No Title,Title away_WCM,Title away_WFM,Title away_WGM,Title away_WIM
0,HÜSI SC,HVSE-Infraplan,1,2374,Böröcz István,1,0,Jankovic Alojzije,2557,2015-11-07,...,0,0,0,1,0,0,0,0,0,0
1,HÜSI SC,HVSE-Infraplan,2,2362,Havasi Gergő,0,1,Ruck Róbert,2565,2015-11-07,...,0,0,0,1,0,0,0,0,0,0
2,HÜSI SC,HVSE-Infraplan,3,2470,Kosic Dragan,1,0,Németh Miklós,2469,2015-11-07,...,0,0,0,0,1,0,0,0,0,0
3,HÜSI SC,HVSE-Infraplan,4,2424,Stankovic Milos,0,1,Dr. Kovács Gábor,2462,2015-11-07,...,0,0,0,0,1,0,0,0,0,0
4,HÜSI SC,HVSE-Infraplan,5,2426,Drazic Sinisa,1,0,Mészáros Michal,2389,2015-11-07,...,0,0,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27062,Félegyháza-Csongrád,Gyulai SE,8,1924,Kelemen Sándor,0,1,Oltean Cristian-Daniel,1911,2020-03-08,...,0,0,0,0,0,1,0,0,0,0
27063,Félegyháza-Csongrád,Gyulai SE,9,1943,Szekeres Sebestyén,1,0,Györgydeák Tibor,1766,2020-03-08,...,0,0,0,0,0,1,0,0,0,0
27064,Félegyháza-Csongrád,Gyulai SE,10,1738,Hideg Ferenc Bendegúz,0,1,Gurzó Bence Bendeguz,1571,2020-03-08,...,0,0,0,0,0,1,0,0,0,0
27065,Félegyháza-Csongrád,Gyulai SE,11,1590,Nagy Noémi,1,0,Vaszkó Szabolcs,1400,2020-03-08,...,1,0,0,0,0,0,0,0,0,0


In [25]:
test['Result_home'] = test['Result_home'].apply(np.float64)
test['Result_away'] = test['Result_away'].apply(np.float64)
test.corrwith(test['Result_home']).sort_values(ascending=False)

Result_home            1.000000
Elo home               0.098952
Title away_-           0.082441
Title away_No Title    0.063351
Title home_GM          0.032040
                         ...   
Elo_away              -0.162867
Elo away              -0.164747
Standard elo_away     -0.202495
Total points_away     -0.368570
Result_away           -1.000000
Length: 68, dtype: float64

In [26]:
df[(df['Youth_home']==1) | (df['Youth_away']==1)]['Youth_away']#.groupby(by=["League_home"])#['Youth_home','Youth_away']#.sum().plot.bar()

11       1.0
23       1.0
29       1.0
30       1.0
34       1.0
        ... 
27054    1.0
27062    1.0
27064    1.0
27065    1.0
27066    1.0
Name: Youth_away, Length: 5120, dtype: float64

In [27]:
df[(df['Female_home']=='1')] #.groupby(by="Season").size().plot()

,Home team,Away team,Board no.,Title home,Elo home,Player home,Color home,Color away,Player away,Elo away,...,2019 Year blitz games total_away,2019 Year standard rating change total_away,2019 Year rapid rating change total_away,2019 Year blitz rating change total_away,2020 Year standard games total_away,2020 Year rapid games total_away,2020 Year blitz games total_away,2020 Year standard rating change total_away,2020 Year rapid rating change total_away,2020 Year blitz rating change total_away
10,HÜSI SC,HVSE-Infraplan,11,WIM,2057,Nagy-Jamalia Natalie,1,0,Némethné Lakos Nikoletta,2275,...,0.0,-21.0,0.0,0.0,7.0,0.0,0.0,-7.0,0.0,0.0
19,Aquaréna Kőbánya SC,Pénzügyőr SE,8,WFM,2271,Marjanovic Annamária,0,1,Szeberényi Ádám,2370,...,7.0,-6.0,0.0,-15.0,75.0,0.0,0.0,-19.0,0.0,0.0
32,Z.Csuti-Hydrocomp SK,Budapesti Titánok,9,IM,2343,Mádl Ildikó,1,0,Juhász Kristóf,2369,...,0.0,55.0,1.0,0.0,17.0,0.0,11.0,3.0,0.0,-67.0
47,Aquaprofit-NTSK I.,Dunaharaszti MTK,12,WGM,2353,Gara Tícia,0,1,Erdős Boglárka,2190,...,0.0,12.0,0.0,0.0,4.0,0.0,0.0,6.0,0.0,0.0
59,ASE Paks,ASS-Makói SVSE,12,IM,2358,Lázárné Vajda Szidónia,0,1,Juhász Barbara,2185,...,0.0,35.0,-2.0,0.0,21.0,13.0,0.0,1.0,23.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26846,Félegyháza-Csongrád,Duocor-Makói SVSE II,11,No Title,1590,Nagy Noémi,1,0,Hajnal Bence,1779,...,0.0,-2.0,0.0,0.0,4.0,0.0,0.0,-37.0,0.0,0.0
26863,Szentesi Sakk SE,TLSE Kecskemét,4,No Title,2042,Nagy Helga,0,1,Jurcsák András,2141,...,0.0,-5.0,0.0,0.0,6.0,0.0,0.0,-1.0,0.0,0.0
26883,Maróczy SE II.,TÁKISZ SE,12,WCM,1643,Márkus Molli Amina,0,1,Zsirmik Zsombor,1348,...,9.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0
26973,Szentesi Sakk SE,Duocor-Makói SVSE II,4,No Title,2042,Nagy Helga,0,1,Martonosi Ádám,1918,...,0.0,23.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0


In [28]:
df.columns.values
df['Youth_home'].dtype

dtype('O')

In [29]:
df.to_csv('ChessDataset.csv', index=False)

In [30]:
s = all_ranklist[all_ranklist['Player name'] == 'Kovács Gergő Patrik']
s['2015 Year standard games total']

3748    -160
7783    -160
12106   -160
16353   -160
20327   -160
Name: 2015 Year standard games total, dtype: int64

In [31]:
df[df['Season_away'] != df['Season_home']]['Season_home']

Series([], Name: Season_home, dtype: object)

In [32]:
df[df['Season_away'] != df['Season_home']]['Season_away']

Series([], Name: Season_away, dtype: object)

In [33]:
all_ranklist['Season']

0        15-16
1        15-16
2        15-16
3        15-16
4        15-16
         ...  
20452    19-20
20453    19-20
20454    19-20
20455    19-20
20456    19-20
Name: Season, Length: 20457, dtype: object

## EDA : Exploratory Data Analysis
Questions:


*   How do youth and female players perform?
*   When do they perform best?
*   Which player exceeded expectations the most?
*   How does nationality affect performance?
*   Does age matter? How much? What makes a young player perform better?
*   Is the any correlation how a player performs before/after a game? E.g. can you expect someone that lost 3 games before to win?
*   How can you predict someone to win? Which factors are the most important?
*   Does distance impact performance?



In [34]:
# load dataset

#df = pd.read_csv('ChessDataset.csv', encoding='utf8')

In [35]:
def finalize_dataframe(df):
    df = df.apply(pd.to_numeric, errors='ignore')
    del df['Season_home']
    del df['League_home']
    del df['League_away']
    return df
df = finalize_dataframe(df)
df

,Home team,Away team,Board no.,Title home,Elo home,Player home,Color home,Color away,Player away,Elo away,...,2019 Year blitz games total_away,2019 Year standard rating change total_away,2019 Year rapid rating change total_away,2019 Year blitz rating change total_away,2020 Year standard games total_away,2020 Year rapid games total_away,2020 Year blitz games total_away,2020 Year standard rating change total_away,2020 Year rapid rating change total_away,2020 Year blitz rating change total_away
0,HÜSI SC,HVSE-Infraplan,1,IM,2374,Böröcz István,1,0,Jankovic Alojzije,2557,...,70.0,-16.0,-46.0,16.0,2.0,9.0,23.0,1.0,20.0,3.0
1,HÜSI SC,HVSE-Infraplan,2,No Title,2362,Havasi Gergő,0,1,Ruck Róbert,2565,...,66.0,17.0,3.0,-52.0,17.0,41.0,67.0,15.0,-85.0,27.0
2,HÜSI SC,HVSE-Infraplan,3,GM,2470,Kosic Dragan,1,0,Németh Miklós,2469,...,16.0,13.0,0.0,-1.0,26.0,8.0,0.0,-8.0,-6.0,0.0
3,HÜSI SC,HVSE-Infraplan,4,IM,2424,Stankovic Milos,0,1,Dr. Kovács Gábor,2462,...,11.0,1.0,0.0,-51.0,7.0,0.0,0.0,0.0,0.0,0.0
4,HÜSI SC,HVSE-Infraplan,5,GM,2426,Drazic Sinisa,1,0,Mészáros Michal,2389,...,0.0,9.0,0.0,0.0,2.0,0.0,0.0,-3.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27062,Félegyháza-Csongrád,Gyulai SE,8,No Title,1924,Kelemen Sándor,0,1,Oltean Cristian-Daniel,1911,...,33.0,147.0,138.0,312.0,13.0,37.0,8.0,195.0,126.0,81.0
27063,Félegyháza-Csongrád,Gyulai SE,9,No Title,1943,Szekeres Sebestyén,1,0,Györgydeák Tibor,1766,...,0.0,26.0,0.0,0.0,9.0,0.0,0.0,-4.0,0.0,0.0
27064,Félegyháza-Csongrád,Gyulai SE,10,No Title,1738,Hideg Ferenc Bendegúz,0,1,Gurzó Bence Bendeguz,1571,...,0.0,7.0,0.0,0.0,5.0,0.0,0.0,34.0,0.0,0.0
27065,Félegyháza-Csongrád,Gyulai SE,11,No Title,1590,Nagy Noémi,1,0,Vaszkó Szabolcs,1400,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [38]:
female_ptable = pd.pivot_table(df, values='Female_home', index=['Season'], columns=['League'], aggfunc=np.sum)
female_ptable

#t = df.apply(pd.to_numeric, errors='ignore')
#t


League,nb1,nb1bchar,nb1bmar,nb2aszt,nb2bar,nb2bre,nb2erk,nb2sze,nb2toth
Season,,,,,,,,,
15-16,45.0,60.0,51.0,25.0,21.0,37.0,15.0,60.0,30.0
16-17,47.0,NaN,50.0,14.0,22.0,24.0,16.0,64.0,34.0
17-18,49.0,54.0,NaN,20.0,28.0,42.0,18.0,35.0,28.0
18-19,44.0,69.0,82.0,19.0,21.0,16.0,5.0,54.0,27.0
19-20,51.0,73.0,83.0,28.0,NaN,18.0,NaN,27.0,9.0


In [40]:
total_ptable = pd.pivot_table(df, index=['Season'], columns=['League'], aggfunc=np.count)
total_ptable

AttributeError: module 'numpy' has no attribute 'count'